# 2.2 Plotting train stations and benches on a map

## 2.2.1 OpenStreetMap

There are some benches, footpaths, plots of grass. In the surroundings it shows residential areas, boundaries and other regions.

There are points, lines, polygons and relations.

Here is a nice bench: Location: 53.1822114, 6.5650417

It even shows part of the "Wandelnetwerk Groningen", cool!

## 2.2.2 Plotting the stations
The osm file is an xml document/dataset with keywords/syntax specific to openstreetmaps

In [2]:
import folium
from folium.plugins import MarkerCluster
from pyrosm import OSM
import pandas as pd
import geopandas as gpd

In [51]:
# Load the data

# Centered around groningen?
minlat=52.837213
minlon=6.166463
maxlat=53.659729
maxlon=7.450069

osm = OSM('railway_data.osm.pbf') #, bounding_box=[minlon, minlat, maxlon, maxlat])

# location = [(minlat+maxlat)/2, (minlon+maxlon)/2]
# location = [53.2194, 6.5665] 
# location = [53.271088, 6.805415]
location = [52.5143247,5.9418464]

stations = osm.get_data_by_custom_criteria(
    custom_filter={'railway': ['station']}, 
    extra_attributes=['name'],
    keep_nodes=True,
    keep_ways=False,
    keep_relations=False
)

tracks = osm.get_data_by_custom_criteria(
    custom_filter={'railway': ['rail']},
    keep_nodes=False,
    keep_ways=True,
    keep_relations=False
)


m = folium.Map(location=location, zoom_start=8)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in stations.iterrows():
    icon = folium.Icon(icon='train', prefix='fa', color='orange')
    name = row['name'] if row['name'] != None else 'No name'
    popup = folium.Popup(name, max_width=len(name) * 10)
    tooltip = folium.Tooltip(name)
    folium.Marker(location=[row['lat'], row['lon']], popup=popup, 
                  tooltip=tooltip, icon=icon).add_to(marker_cluster)


for idx, row in tracks.iterrows():
    geom = row['geometry']
    if geom.geom_type == 'LineString':
        folium.PolyLine(locations=[(coord[1], coord[0]) for coord in geom.coords]).add_to(m)
    elif geom.geom_type == 'MultiLineString':
        for linestring in geom.geoms:
            folium.PolyLine(locations=[(coord[1], coord[0]) for coord in linestring.coords]).add_to(m)


m.save('nl_railway.html')

Downloaded groningen-latest.oms.pbf and ran:
```
osmium tags-filter groningen-latest.osm.pbf n/amenity=bench -o groningen-benches.osm.pbf --overwrite
```

In [19]:
bench_locations = gpd.read_file('groningen-benches.osm.pbf', layer='points')['geometry']
m = folium.Map(location=[53.2194, 6.5665], zoom_start=10)

cluster = MarkerCluster().add_to(m)

for point in bench_locations:
    icon = folium.Icon(icon='chair', prefix='fa', color='green')
    folium.Marker(location=(point.y, point.x), icon=icon).add_to(cluster)

m.save('groningen_benches.html')

In [20]:
from folium.plugins import HeatMap
HeatMap(zip(bench_locations.y.values, bench_locations.x.values)).add_to(m)
folium.LayerControl().add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
m.save('groningen_benches_heatmap.html')